### Пайплайн для стекинга моделей

In [1]:
import pandas as pd
from sklearn import metrics
import numpy as np

In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import random
import warnings

warnings.simplefilter('ignore')

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

random.seed(42)
np.random.seed(42)

In [46]:
main = pd.read_csv('../data/train.csv')
real = pd.read_csv('../data/sample_submit_naive.csv').drop('predict', axis=1)
real['target'] = -1
main = pd.concat([main, real])

In [41]:
test_ids = pd.read_csv('../data/test_ids.csv')
train_, test_ = main.loc[~(main['user_id'].isin(test_ids['user_id'])) & ~(main['user_id'].isin(real['user_id']))], main.loc[main['user_id'].isin(test_ids['user_id'])]

In [44]:
agg = pd.read_csv('../predictions/agg_pred_meta.csv')
agg_1 = pd.read_csv('../predictions/agg_pred_new_meta.csv')
agg_2 = pd.read_csv('../predictions/agg_pred_meta_best.csv')
agg_3 = pd.read_csv('../predictions/agg_pred_meta_ensemble.csv')
agg_ens = pd.read_csv('../predictions/pred_newagg_meta.csv')

time_2 = pd.read_csv('../predictions/time_pred_newagg_800_alfabetanew_meta.csv')
time_3 = pd.read_csv('../predictions/time_pred_all_data_meta_newsfold2.csv')

emb = pd.read_csv('../predictions/emb_pred_meta_new_ab.csv')
emb_1 = pd.read_csv('../predictions/emb_pred_meta_clusters.csv')
emb_2 = pd.read_csv('../predictions/emb_ansemble_pred_meta.csv')
emb_3 = pd.read_csv('../predictions/emb_ansemble_pred_meta_30.csv')
emb_3 = emb_3.drop(['Unnamed: 0'], axis=1)
emb_4 = pd.read_csv('../predictions/emb_ansemble_pred_clusters_meta.csv')
emb_5 = pd.read_csv('../predictions/emb_ansemble_pred_clusters_new_meta.csv')
emb_6 = pd.read_csv('../predictions/emb_ansemble_pred_meta_6_10.csv')

emb_5 = emb_5.drop('Unnamed: 0', axis=1)
emb_4 = emb_4.drop('Unnamed: 0', axis=1)

time = pd.read_csv('../predictions/time_pred_meta_new_embs.csv')
time_14 = pd.read_csv('../predictions/agg_time_pred_meta.csv')
time_15 = pd.read_csv('../predictions/agg_time_pred_meta_new.csv')

trx_agg = pd.read_csv('../predictions/agg_trx_pred_meta.csv')
tfidf = pd.read_csv('../predictions/agg_new_group_tf_idfpred_meta.csv')

supervised = pd.read_csv('../predictions/sp-preds_train-test.csv')
supervised_wtte = pd.read_csv('../sp-preds_train-test-2.csv')

In [ ]:
main = main.merge(agg, on='user_id')
main = main.merge(agg_1, on='user_id')
main = main.merge(agg_2, on='user_id')
main = main.merge(agg_3, on='user_id')

main = main.merge(emb, on='user_id')
main = main.merge(emb_1, on='user_id')
main = main.merge(emb_2, on='user_id')
main = main.merge(emb_3, on='user_id')
main = main.merge(emb_4, on='user_id')
main = main.merge(emb_5, on='user_id')
main = main.merge(emb_6, on='user_id')


main = main.merge(time, on='user_id')
main = main.merge(time_2, on='user_id')
main = main.merge(trx_agg, on='user_id')

main = main.merge(tfidf, on='user_id')

main = main.merge(supervised, on='user_id')
main = main.drop(['Unnamed: 0'], axis=1)
main['sp'] =  (main['sp'] - main['sp'].min()) / (main['sp'].max() - main['sp'].min())
main

In [625]:
train = train_.merge(main, on='user_id')
test = test_.merge(main, on='user_id')

,user_id,target,time,pred_agg_x,pred_agg_new,pred_agg_best,pred_agg_ans,pred_emb_x,pred_emb_y,pred_emb_sum_x,pred_emb_sum_y,pred_emb_sum_x,pred_emb_sum_y,pred_emb_sum,pred_time_x,pred_time_y,pred_agg_y,pred_agg,sp
0,379659,0,91,0.043767,0.046361,0.042706,0.073616,0.041035,0.027490,0.044168,0.043662,0.044831,0.046517,0.044143,0.029024,0.022356,0.042674,0.039489,0.060161
1,317004,0,90,0.027901,0.016313,0.029536,0.035834,0.015389,0.014841,0.016746,0.012406,0.015769,0.013962,0.013580,0.020668,0.009771,0.022729,0.009215,0.007400
2,329546,0,67,0.085214,0.124443,0.077833,0.059111,0.057007,0.063897,0.092479,0.070766,0.066270,0.069061,0.095701,0.063020,0.058900,0.062108,0.092936,0.041237
3,16918,0,89,0.058912,0.072740,0.052820,0.077643,0.091741,0.079662,0.087437,0.085124,0.089968,0.082101,0.093041,0.064527,0.064850,0.079564,0.054910,0.072529
4,246698,0,91,0.089005,0.099994,0.095070,0.083390,0.072847,0.046739,0.080446,0.102000,0.120165,0.133263,0.106655,0.052172,0.056897,0.040486,0.128062,0.043409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51195,538772,0,75,0.242358,0.190385,0.272205,0.123460,0.164817,0.096858,0.239673,0.163432,0.150706,0.138711,0.153955,0.216282,0.082213,0.220565,0.221046,0.152432
51196,330596,0,91,0.047577,0.024738,0.041630,0.043064,0.034313,0.038120,0.031054,0.035787,0.039825,0.040205,0.038454,0.024461,0.021177,0.032710,0.044346,0.037365
51197,7334,0,91,0.022756,0.016812,0.022727,0.039632,0.031047,0.026606,0.030981,0.032217,0.031923,0.030218,0.028018,0.034384,0.031303,0.029428,0.022701,0.036914
51198,134987,0,91,0.044377,0.057428,0.043870,0.049514,0.072681,0.092147,0.072933,0.078723,0.079082,0.080269,0.084136,0.039636,0.039061,0.054311,0.054650,0.111979


In [627]:
X_train, y_train = train.drop(['target', 'user_id', 'time'], axis=1), train['target']
X_train

,pred_agg_x,pred_agg_new,pred_agg_best,pred_agg_ans,pred_emb_x,pred_emb_y,pred_emb_sum_x,pred_emb_sum_y,pred_emb_sum_x,pred_emb_sum_y,pred_emb_sum,pred_time_x,pred_time_y,pred_agg_y,pred_agg,sp
0,0.043767,0.046361,0.042706,0.073616,0.041035,0.027490,0.044168,0.043662,0.044831,0.046517,0.044143,0.029024,0.022356,0.042674,0.039489,0.060161
1,0.027901,0.016313,0.029536,0.035834,0.015389,0.014841,0.016746,0.012406,0.015769,0.013962,0.013580,0.020668,0.009771,0.022729,0.009215,0.007400
2,0.085214,0.124443,0.077833,0.059111,0.057007,0.063897,0.092479,0.070766,0.066270,0.069061,0.095701,0.063020,0.058900,0.062108,0.092936,0.041237
3,0.058912,0.072740,0.052820,0.077643,0.091741,0.079662,0.087437,0.085124,0.089968,0.082101,0.093041,0.064527,0.064850,0.079564,0.054910,0.072529
4,0.089005,0.099994,0.095070,0.083390,0.072847,0.046739,0.080446,0.102000,0.120165,0.133263,0.106655,0.052172,0.056897,0.040486,0.128062,0.043409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51195,0.242358,0.190385,0.272205,0.123460,0.164817,0.096858,0.239673,0.163432,0.150706,0.138711,0.153955,0.216282,0.082213,0.220565,0.221046,0.152432
51196,0.047577,0.024738,0.041630,0.043064,0.034313,0.038120,0.031054,0.035787,0.039825,0.040205,0.038454,0.024461,0.021177,0.032710,0.044346,0.037365
51197,0.022756,0.016812,0.022727,0.039632,0.031047,0.026606,0.030981,0.032217,0.031923,0.030218,0.028018,0.034384,0.031303,0.029428,0.022701,0.036914
51198,0.044377,0.057428,0.043870,0.049514,0.072681,0.092147,0.072933,0.078723,0.079082,0.080269,0.084136,0.039636,0.039061,0.054311,0.054650,0.111979


In [628]:
from sklearn import linear_model
reg = linear_model.Ridge(alpha=12.85,
                         random_state = 42,
                        )
reg.fit(X_train, y_train)

Ridge(alpha=12.85, random_state=42)

In [629]:
y_pred = reg.predict(test.drop(['target', 'user_id', 'time'], axis=1))y_pred.shape

(12800,)

In [622]:
coef = reg.coef_
coef

array([-0.07461034,  0.04723893, -0.08173178,  0.02875297, -0.1489052 ,
       -0.03122304,  0.05542683,  0.09159869,  0.05981415,  0.03038276,
        0.02086542,  0.21318692,  0.12588865,  0.3656878 ,  0.20033574,
        0.15467096])

In [630]:
print(metrics.roc_auc_score(test['target'], y_pred))

0.7932362935468447


In [631]:
from lifelines.utils import concordance_index

concordance_index(test['time'], -y_pred, test['target'])

0.7943150613108246

In [633]:
real = real.merge(main, on='user_id')

In [638]:
y_pred = reg.predict(real.drop(['user_id'], axis=1))
y_pred.shape

(32000,)

In [603]:
predict.shape

(12800,)

In [639]:
# Финальный предикт
pred = {'user_id': real.user_id.values, 'predict': y_pred}

pd.DataFrame(pred).to_csv('submission.csv', index=False)